[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oreilly-japan/RecommenderSystems/blob/main/chapter5/colab/Word2vec.ipynb)

# Word2vec

In [1]:
# Colab용 notebook입니다. 이 notebook 한 장에서 여러 데이터의 다운로드부터, 추천까지 완결하도록 되어 있습니다(예측 평가는 미포함)
# MovieLens 데이터를 아직 다운로드 하지 않았다면, 이 셀을 실행해서 다운로드합니다.
# MovieLens 데이터 분석은 data_download.ipynb를 참조합니다.

# 데이터 다운로드와 압축 풀기
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data
!unzip -n ../data/ml-10m.zip -d ../data/

--2022-12-27 05:54:34--  https://files.grouplens.org/datasets/movielens/ml-10m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65566137 (63M) [application/zip]
Saving to: ‘../data/ml-10m.zip’

ml-10m.zip          100%[===================>]  62.53M   115MB/s    in 0.5s    

2022-12-27 05:54:35 (115 MB/s) - ‘../data/ml-10m.zip’ saved [65566137/65566137]

Archive:  ../data/ml-10m.zip
   creating: ../data/ml-10M100K/
  inflating: ../data/ml-10M100K/allbut.pl  
  inflating: ../data/ml-10M100K/movies.dat  
  inflating: ../data/ml-10M100K/ratings.dat  
  inflating: ../data/ml-10M100K/README.html  
  inflating: ../data/ml-10M100K/split_ratings.sh  
  inflating: ../data/ml-10M100K/tags.dat  


In [2]:
# Movielens 데이터 로딩(데이터량이 많으므로, 로딩에 시간이 걸릴 수 있습니다)
import pandas as pd

# movieID와 제목만 사용
m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv('../data/ml-10M100K/movies.dat', names=m_cols, sep='::' , encoding='latin-1', engine='python')

# genre를 list 형식으로 저장한다
movies['genre'] = movies.genre.apply(lambda x:x.split('|'))


# 사용자가 부여한 영화의 태그 정보를 로딩한다
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv('../data/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')

# tag를 소문자로 바꾼다
user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()


# tag를 영화별로 list 형식으로 저장한다
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list})

# 태그 정보를 결합한다
movies = movies.merge(movie_tags, on='movie_id', how='left')

# 평갓값 데이터만 로딩한다
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/ml-10M100K/ratings.dat', names=r_cols, sep='::', engine='python')


# 데이터량이 많으므로 사용자수를 1000으로 줄여서 시험해본다
valid_user_ids = sorted(ratings.user_id.unique())[:1000]
ratings = ratings[ratings["user_id"].isin(valid_user_ids)]


# 영화 데이터와 평가 데이터를 결합한다
movielens = ratings.merge(movies, on='movie_id')

print(f'unique_users={len(movielens.user_id.unique())}, unique_movies={len(movielens.movie_id.unique())}')

# 학습용과 데이터용으로 데이터를 나눈다
# 각 사용자의 최근 5건의 영화를 평가용으로 사용하고, 나머지는 학습용으로 사용한다
# 우선, 각 사용자가 평가한 영화의 순서를 계산한다
# 최근 부여한 영화부터 순서를 부여한다(1에서 시작)

movielens['timestamp_rank'] = movielens.groupby(
    'user_id')['timestamp'].rank(ascending=False, method='first')
movielens_train = movielens[movielens['timestamp_rank'] > 5]
movielens_test = movielens[movielens['timestamp_rank']<= 5]

unique_users=1000, unique_movies=6736


In [3]:
# 인자 수
factors = 100
# 에폭 수
n_epochs = 30
# window 크기
window = 100
# 스킵 그램
use_skip_gram = 1
# 계층적 소프트맥스
use_hierarchial_softmax = 0
# 사용할 단어의 출현 횟수의 임곗값
min_count = 5

In [4]:
movie_content = movies.copy()
# tag가 부여되지 않은 영화는 있지만, genre는 모든 영화에 부여되어 있다
# tag와 genre를 결합한 것을 영화의 콘텐츠 정보로 해서 비슷한 영화를 찾아 추천한다
# tag가 없는 영화는 NaN으로 되어 있으므로, 빈 리스트로 변환한 뒤 처리한다
movie_content["tag_genre"] = movie_content["tag"].fillna("").apply(list) + movie_content["genre"].apply(list)
movie_content["tag_genre"] = movie_content["tag_genre"].apply(lambda x: set(map(str, x)))

In [5]:
!pip install gensim==4.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 23.9 MB 1.2 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [6]:
import gensim

# 태그와 장르 데이터를 사용해 word2vec을 학습한다
tag_genre_data = movie_content.tag_genre.tolist()
model = gensim.models.word2vec.Word2Vec(
    tag_genre_data,
    vector_size=factors,
    window=window,
    sg=use_skip_gram,
    hs=use_hierarchial_softmax,
    epochs=n_epochs,
    min_count=min_count,
)


/usr/local/lib/python3.8/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [7]:
# anime 태크와 비슷한 태그를 확인한다
model.wv.most_similar('anime')

[('studio ghibli', 0.8121964335441589),
 ('zibri studio', 0.7996001243591309),
 ('miyazaki', 0.7846846580505371),
 ('pelicula anime', 0.7555471658706665),
 ('hayao miyazaki', 0.7477614879608154),
 ('japan', 0.6425853967666626),
 ('Animation', 0.5504586696624756),
 ('curse', 0.5254901647567749),
 ('steampunk', 0.5219841599464417),
 ('dragon', 0.5078659057617188)]

In [8]:
# 각 영화의 벡터를 계산한다
# 각 영화에 부여되어 있는 태그/장르 벡터의 평균을 영화의 벡터로 한다
movie_vectors = []
tag_genre_in_model = set(model.wv.key_to_index.keys())

titles = []
ids = []

for i, tag_genre in enumerate(tag_genre_data):
    # word2vec 모델에서 사용할 수 있는 태그/장르로 필터링한다
    input_tag_genre = set(tag_genre) & tag_genre_in_model
    if len(input_tag_genre) == 0:
        # word2vec에 기반해 벡터 계산할 수 없는 영화에는 무작위 벡터를 부여한다
        vector = np.random.randn(model.vector_size)
    else:
        vector = model.wv[input_tag_genre].mean(axis=0)
    titles.append(movie_content.iloc[i]["title"])
    ids.append(movie_content.iloc[i]["movie_id"])
    movie_vectors.append(vector)


In [9]:
import numpy as np

# 후속 유사도 계산을 쉽게 할 수 있도록 numpy 배열로 저장한다
movie_vectors = np.array(movie_vectors)

# 정규화한 벡터
sum_vec = np.sqrt(np.sum(movie_vectors ** 2, axis=1))
movie_norm_vectors = movie_vectors / sum_vec.reshape((-1, 1))


In [10]:
# 아직 평가하지 않은 영화에서 유사도가 높은 아이템을 반환하는 함수
def find_similar_items(vec, evaluated_movie_ids, topn=10):
    score_vec = np.dot(movie_norm_vectors, vec)
    similar_indexes = np.argsort(-score_vec)
    similar_items = []
    for similar_index in similar_indexes:
        if ids[similar_index] not in evaluated_movie_ids:
            similar_items.append(ids[similar_index])
        if len(similar_items) == topn:
            break
    return similar_items

In [11]:
# 사용자가 평갓값 4 이상을 부여한 영화
movielens_train_high_rating = movielens_train[movielens_train.rating >= 4]
user_evaluated_movies = movielens_train.groupby("user_id").agg({"movie_id": list})["movie_id"].to_dict()

id2index = dict(zip(ids, range(len(ids))))
pred_user2items = dict()
for user_id, data in movielens_train_high_rating.groupby("user_id"):
    evaluated_movie_ids = user_evaluated_movies[user_id]
    movie_ids = data.sort_values("timestamp")["movie_id"].tolist()[-5:]

    movie_indexes = [id2index[id] for id in movie_ids]
    user_vector = movie_norm_vectors[movie_indexes].mean(axis=0)
    recommended_items = find_similar_items(user_vector, evaluated_movie_ids, topn=10)
    pred_user2items[user_id] = recommended_items
pred_user2items

{1: [4306, 2054, 7369, 5500, 5452, 2080, 2470, 6559, 36708, 44399],
 2: [1198, 1196, 1097, 3578, 1204, 2028, 7153, 3471, 1287, 527],
 3: [27797, 3874, 61071, 3194, 6384, 58494, 1415, 60494, 4766, 45707],
 4: [2125, 1073, 1441, 551, 1028, 4184, 7834, 837, 4306, 2953],
 5: [1276, 3504, 3168, 2729, 928, 1193, 3741, 2730, 922, 1161],
 6: [5445, 198, 541, 292, 5881, 1214, 2594, 1240, 1129, 1690],
 7: [2935, 955, 4357, 922, 947, 6840, 3307, 946, 950, 7080],
 8: [33437, 517, 3269, 4956, 2764, 36529, 7386, 49007, 5027, 5686],
 9: [2890, 665, 3168, 319, 1162, 1953, 1202, 33166, 4399, 2542],
 10: [1950, 1953, 3504, 7215, 2396, 920, 1230, 1925, 912, 858],
 11: [1784, 339, 3259, 1393, 1307, 7, 1597, 11, 3893, 1101],
 12: [1805, 1438, 517, 2594, 51709, 3510, 1597, 8585, 1061, 8963],
 13: [1784, 1084, 1276, 1230, 356, 1213, 909, 920, 858, 1953],
 14: [586, 3489, 2804, 8516, 3988, 2125, 2161, 48082, 2054, 1894],
 16: [2288, 5522, 5881, 674, 748, 3699, 1274, 5046, 2528, 1301],
 17: [1253, 1254, 3551, 

In [12]:
# user_id=2인 사용자가 학습 데이터에서 4 이상의 평가를 부여한 영화 목록
movielens_train_high_rating[movielens_train_high_rating.user_id==2]

,user_id,movie_id,rating,timestamp,title,genre,tag,timestamp_rank
4732,2,110,5.0,868245777,Braveheart (1995),"[Action, Drama, War]","[bullshit history, medieval, bloodshed, hero, ...",8.0
5246,2,260,5.0,868244562,Star Wars: Episode IV - A New Hope (a.k.a. Sta...,"[Action, Adventure, Sci-Fi]","[desert, quotable, lucas, gfei own it, seen mo...",17.0
5798,2,590,5.0,868245608,Dances with Wolves (1990),"[Adventure, Drama, Western]","[afi 100, lame, native, biopic, american india...",11.0
8381,2,1210,4.0,868245644,Star Wars: Episode VI - Return of the Jedi (1983),"[Action, Adventure, Sci-Fi]","[desert, fantasy, sci-fi, space, lucas, gfei o...",10.0


In [13]:
# user_id=2에 대한 추천(1198, 1196, 1097)
movies[movies.movie_id.isin([1198, 1196, 1097])]

,movie_id,title,genre,tag
1075,1097,E.T. the Extra-Terrestrial (1982),"[Children, Drama, Sci-Fi]","[speilberg, steven spielberg, spielberg/lucas,..."
1171,1196,Star Wars: Episode V - The Empire Strikes Back...,"[Action, Adventure, Sci-Fi]","[lucas, george lucas, george lucas, gfei own i..."
1173,1198,Raiders of the Lost Ark (Indiana Jones and the...,"[Action, Adventure]","[egypt, lucas, seen more than once, dvd collec..."
